In [5]:
with open('../example_op_file.txt', 'r') as f:
    operations = f.readlines()
    op,i1,i2,output = operations[i].split(' ')
    print_instruction(op, i1, i2, output)
    
print_instruction(op, i1, i2, output)
    

NameError: name 'i' is not defined